# Convert Ehost annotation to MedSpacy Document Object
In this note book, we read in annotations and build SpanGroup from overlapped spans. We also build dataframe from annotation for building up OHDSI format database later.

In [1]:
import os, medspacy_io
from pathlib import Path
doc_file = '../mimic_annotations/testEhostReaderData/corpus/18305.txt'
anno_file = '../mimic_annotations/testEhostReaderData/saved/18305.txt.knowtator.xml'
schema_file = '../mimic_annotations/testEhostReaderData/config/projectschema.xml'

In [2]:
print(Path(doc_file).read_text())


Admission Date:  [**2180-9-25**]     Discharge Date:  [**2180-9-29**]

Date of Birth:                  Sex:  F

Service:  MICU

HISTORY OF PRESENT ILLNESS:  Ms. [**Known lastname 50463**] was an 83-year-old
female with a history of polymyalgia rheumatica,
hypercholesterolemia, hypothyroidism, vertigo, postural
hypotension, and a history of syncope in the past who now is
on Florinef and presented to [**Hospital1 **] [**Location (un) 620**]
emergency department on [**2180-9-22**] after a syncopal episode
at home.  She reported that she passed out after urinating
while on the toilet.  She awoke and called her primary care
physician, [**Name10 (NameIs) 1023**] evaluated her in the office and suspected
dehydration, rehydrated her with fluids, and sent her home.
At home she continued to feel poorly, and her primary care
physician told her to return to the Emergency Department.  At
[**Location (un) 620**] emergency department on [**2180-9-23**] EKG revealed
polymorphic ventricular tachycardia

In [3]:
print(Path(anno_file).read_text())

<?xml version="1.0" encoding="UTF-8"?>
<annotations textSource="18305.txt">
    <annotation>
        <mention id="EHOST_Instance_23280" />
        <annotator id="eHOST_2010">sjl</annotator>
        <span start="128" end="2616" />
        <spannedText>HISTORY OF PRESENT ILLNESS: Ms. [**Known lastname 50463**] was an 83-year-old female with a history of polymyalgia rheumatica, hypercholesterolemia, hypothyroidism, vertigo, postural hypotension, and a history of syncope in the past who now is on Florinef and presented to [**Hospital1 **] [**Location (un) 620**] emergency department on [**2180-9-22**] after a syncopal episode at home. She reported that she passed out after urinating while on the toilet. She awoke and called her primary care physician, [**Name10 (NameIs) 1023**] evaluated her in the office and suspected dehydration, rehydrated her with fluids, and sent her home. At home she continued to feel poorly, and her primary care physician told her to return to the Emergency Departme

# eHost Doc Reader

In [4]:
from spacy.lang.en import English
from medspacy_io.reader import EhostDocReader
from medspacy_io.reader import EhostDirReader

## Remark:
1. if you are using `support_overlap=True` the spans/annotations will end up in `doc._.concepts` , if you are using `support_overlap=False` they're in `doc.ents`. In our case it is `support_overlap=True`.
2. `EhostDocReader` will search default folder according to the directory of data. The data and annotations have to be organized as eHost Project:
    1. schema file: `../config/projectsschema.xml`
    2. data folder: `../corpus/*.txt`
    3. annotation folder: `../saved/*.txt.knowtator.xml`

In [5]:
ereader = EhostDocReader(nlp=English(), schema_file=schema_file,support_overlap=True)
print(schema_file,doc_file)
doc = ereader.read(doc_file)
doc._.concepts

../mimic_annotations/testEhostReaderData/config/projectschema.xml ../mimic_annotations/testEhostReaderData/corpus/18305.txt


OrderedDict([('Symptom_Section',
              [HISTORY OF PRESENT ILLNESS:  Ms. [**Known lastname 50463**] was an 83-year-old
               female with a history of polymyalgia rheumatica,
               hypercholesterolemia, hypothyroidism, vertigo, postural
               hypotension, and a history of syncope in the past who now is
               on Florinef and presented to [**Hospital1 **] [**Location (un) 620**]
               emergency department on [**2180-9-22**] after a syncopal episode
               at home.  She reported that she passed out after urinating
               while on the toilet.  She awoke and called her primary care
               physician, [**Name10 (NameIs) 1023**] evaluated her in the office and suspected
               dehydration, rehydrated her with fluids, and sent her home.
               At home she continued to feel poorly, and her primary care
               physician told her to return to the Emergency Department.  At
               [**Location 

In [6]:
doc_concepts_dic = doc._.concepts # doc is doc object readin by eHostReader
print(doc_concepts_dic)
annot_classes = doc_concepts_dic.keys()
print(annot_classes)
print(len(doc_concepts_dic['Symptom_Section']))


#doc.text
#re.finditer(testText,doc.text)
#for match in re.finditer(doc.text, doc_concepts_dic['Symptom_Section'][0]):
 #   print(match.start(),match.end())



OrderedDict([('Symptom_Section', [HISTORY OF PRESENT ILLNESS:  Ms. [**Known lastname 50463**] was an 83-year-old
female with a history of polymyalgia rheumatica,
hypercholesterolemia, hypothyroidism, vertigo, postural
hypotension, and a history of syncope in the past who now is
on Florinef and presented to [**Hospital1 **] [**Location (un) 620**]
emergency department on [**2180-9-22**] after a syncopal episode
at home.  She reported that she passed out after urinating
while on the toilet.  She awoke and called her primary care
physician, [**Name10 (NameIs) 1023**] evaluated her in the office and suspected
dehydration, rehydrated her with fluids, and sent her home.
At home she continued to feel poorly, and her primary care
physician told her to return to the Emergency Department.  At
[**Location (un) 620**] emergency department on [**2180-9-23**] EKG revealed
polymorphic ventricular tachycardia with rate in the 130s and
blood pressure in the 140s to 150s/60s.  She was afebrile at
this t

In [14]:
#doc.spans(doc_concepts_dic['Symptom_Section'])
testSpan = doc_concepts_dic['Symptom_Section'][1]
type(testSpan)
print(testSpan.text)
print(testSpan.label_)
print(testSpan.has_extension)
print(testSpan.start_char, testSpan.end_char)

BRIEF HOSPITAL COURSE:  Ms. [**Known lastname 50463**] was transferred to [**Hospital1 18**]
on [**2180-9-25**].  The initial feeling was that she most likely
was in pulmonary edema, leading to poor oxygenation due to
aggressive fluid and electrolyte replacement at [**Location (un) 620**].  She
was ........... diuresed, and was successfully extubated on
[**2180-9-27**], however, became febrile with known MSSA in her
sputum, however, was persistently febrile despite treatment
with Levofloxacin, Vancomycin, and Flagyl.  Call for concern
of aspiration pneumonia.  She also had an elevated white
count with evidence of bandemia.

On [**2180-9-29**] she also developed a decreased urine output in
the setting of febrile illness, and there was concern for
sepsis.  Her urine output continued to climb despite fluid
boluses.  Her volume status was unclear but most likely
representative of total body fluid overload with decreased
intervascular volume, and plans were made for placement of a
PA cathet

# Build SpanGroup

In [9]:
from spacy.tokens import SpanGroup
#doc.spans["errors"] = spans
group = SpanGroup(doc, name="Symptom_Section", spans=doc_concepts_dic['Symptom_Section'])
doc.spans["Symptom_Section"] = group


In [10]:
type(doc_concepts_dic['Symptom_Section'][0])
type(doc.spans["Symptom_Section"])
print(len(doc.spans["Symptom_Section"]))
assert doc.spans["Symptom_Section"][0]==doc_concepts_dic['Symptom_Section'][0] #the first element in spanGroup is the first element in span list
print(doc.spans["Symptom_Section"].has_overlap,doc.spans["Symptom_Section"].__len__)
#doc_concepts_dic['Symptom_Section'][0].span_txt

2
False <method-wrapper '__len__' of spacy.tokens.span_group.SpanGroup object at 0x7fca5021a9a0>


# Build OHDSI format database from annotations

This is based on https://github.com/OHDSI/CommonDataModel/blob/main/inst/ddl/5.4/sql_server/OMOPCDM_sql_server_5.4_ddl.sql

In [13]:
for concept,listSpan in doc_concepts_dic.items():
    print(concept, len(listSpan),listSpan)

Symptom_Section 2 [HISTORY OF PRESENT ILLNESS:  Ms. [**Known lastname 50463**] was an 83-year-old
female with a history of polymyalgia rheumatica,
hypercholesterolemia, hypothyroidism, vertigo, postural
hypotension, and a history of syncope in the past who now is
on Florinef and presented to [**Hospital1 **] [**Location (un) 620**]
emergency department on [**2180-9-22**] after a syncopal episode
at home.  She reported that she passed out after urinating
while on the toilet.  She awoke and called her primary care
physician, [**Name10 (NameIs) 1023**] evaluated her in the office and suspected
dehydration, rehydrated her with fluids, and sent her home.
At home she continued to feel poorly, and her primary care
physician told her to return to the Emergency Department.  At
[**Location (un) 620**] emergency department on [**2180-9-23**] EKG revealed
polymorphic ventricular tachycardia with rate in the 130s and
blood pressure in the 140s to 150s/60s.  She was afebrile at
this time.

Labs in t

In [15]:
import pandas as pd
data = []
for concept, listSpan in doc_concepts_dic.items():
    for sp in listSpan:
        dicElem = {}
        dicElem['Span Text'] = sp.text
        dicElem['Concept Label'] = sp.label_
        dicElem['start loc'] = sp.start_char
        dicElem['end loc'] = sp.end_char
        dicElem['doc name'] = '18305.txt'
        
    data.append(dicElem)
        
   
print(data)

[{'Span Text': 'BRIEF HOSPITAL COURSE:  Ms. [**Known lastname 50463**] was transferred to [**Hospital1 18**]\non [**2180-9-25**].  The initial feeling was that she most likely\nwas in pulmonary edema, leading to poor oxygenation due to\naggressive fluid and electrolyte replacement at [**Location (un) 620**].  She\nwas ........... diuresed, and was successfully extubated on\n[**2180-9-27**], however, became febrile with known MSSA in her\nsputum, however, was persistently febrile despite treatment\nwith Levofloxacin, Vancomycin, and Flagyl.  Call for concern\nof aspiration pneumonia.  She also had an elevated white\ncount with evidence of bandemia.\n\nOn [**2180-9-29**] she also developed a decreased urine output in\nthe setting of febrile illness, and there was concern for\nsepsis.  Her urine output continued to climb despite fluid\nboluses.  Her volume status was unclear but most likely\nrepresentative of total body fluid overload with decreased\nintervascular volume, and plans were m

In [16]:
df = pd.DataFrame(data)
df

,Span Text,Concept Label,start loc,end loc,doc name
0,BRIEF HOSPITAL COURSE: Ms. [**Known lastname ...,Symptom_Section,2618,4272,18305.txt
1,BRIEF HOSPITAL COURSE:,SectionHeader_HasSymptom,2618,2640,18305.txt
2,cardiac arrest,Symptom,4257,4271,18305.txt
